In [97]:
import glob
import numpy as np
csv_files = (glob.glob("C:/Users/Vishnu Kumar/Documents/IPL/raw/*info.csv")) 

In [98]:
import pandas as pd


df_append = pd.DataFrame()

for file in csv_files:
            df_temp = pd.read_csv(file,sep='漢ழ்字', engine='python',header=None)
            df_temp['ID'] = file
            df_temp = df_temp[(df_temp[0].str.startswith('info,toss_winner')) | (df_temp[0].str.startswith('info,toss_decision'))]
            df_append = pd.concat([df_append,df_temp])

df_append = df_append.reset_index()
df_append[['info', 'attrib','value']] = df_append[0].str.split(",", expand = True)
df_append = df_append.drop(columns=['index',0,'info'])
df_append['ID'] = df_append['ID'].str[40:-9]
toss_winner = df_append[df_append['attrib']=='toss_winner'].copy()
toss_decision = df_append[df_append['attrib']=='toss_decision'].copy()
toss_winner['ID'] = toss_winner['ID'].astype('int')
toss_decision['ID'] = toss_decision['ID'].astype('int')

In [99]:
all_matches = pd.read_csv("C:/Users/Vishnu Kumar/Documents/IPL/raw/all_matches.csv")
# all_matches
all_matches = pd.merge(all_matches, toss_winner, how="left" ,left_on = "match_id", right_on = "ID" )
all_matches = all_matches.drop(columns=["ID", "attrib"])
all_matches = all_matches.rename(columns={"value": "toss_winner"})
all_matches = pd.merge(all_matches, toss_decision, how="left" ,left_on = "match_id", right_on = "ID" )
all_matches = all_matches.drop(columns=["ID", "attrib"])
all_matches = all_matches.rename(columns={"value": "toss_decision"})
all_matches = all_matches[all_matches['innings']<=2]
all_matches['season'] = all_matches['season'].replace(['2007/08', '2009', '2009/10', '2011' ,'2012','2018', '2019', '2020/21', '2021'], [2008,2009,2010,2011,2012,2018,2019,2020,2021])
all_matches['toss_win_batting'] = all_matches.apply(lambda x: 1 if x['toss_winner'] == x['batting_team'] else 0, axis=1)
all_matches['toss_decision'] = all_matches.apply(lambda x: 1 if x['toss_decision'] == 'bat' else 0, axis=1)
all_matches['player_dismissed'] = all_matches.apply(lambda x: 1 if x['player_dismissed'] == x['striker'] else 0, axis=1)
all_data = all_matches.copy()
# all_matches = pd.concat([all_matches, toss_winner], axis=1, join='outer', join_axes=[match_id,ID])

C:\Users\Vishnu Kumar\AppData\Local\Temp\ipykernel_15892\579395103.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  all_matches = pd.read_csv("C:/Users/Vishnu Kumar/Documents/IPL/raw/all_matches.csv")


In [100]:
all_matches['ball'] = (all_matches['ball'].astype('int')*6)+((all_matches['ball'] - all_matches['ball'].astype('int'))*10)

In [101]:
all_matches['wicket'] = all_matches['wicket_type'].notnull().astype(int)

In [102]:
all_matches = all_matches.fillna(0)

In [103]:
all_matches['byes'] = all_matches['byes'] + all_matches['legbyes']
all_matches['extras'] = all_matches['extras'] - all_matches['byes']

In [104]:
all_matches = all_matches.drop(columns=['match_id','season','start_date','batting_team','non_striker','bowling_team','innings','player_dismissed','other_wicket_type','toss_winner','toss_decision','toss_win_batting','wides','noballs','legbyes','penalty','other_player_dismissed','wicket_type'])

In [105]:
# all_matches.info()
all_matches['output'] = all_matches.apply(lambda x: 'W' if x['wicket'] == 1 else str(int(x['runs_off_bat']+x['extras']+x['byes'])), axis=1)

In [106]:
all_matches = all_matches.drop(columns=['runs_off_bat','extras','byes','wicket'])

In [109]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

# Load the dataset into a pandas DataFrame
df = all_matches.copy()

# Use LabelEncoder to convert categorical variables to numerical variables
le = LabelEncoder()
df['venue'] = le.fit_transform(df['venue'])
df['striker'] = le.fit_transform(df['striker'])
df['bowler'] = le.fit_transform(df['bowler'])

# Split the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2)

# Create input and output variables for the model
X_train = train_df[['venue', 'ball', 'striker', 'bowler']]
y_train = train_df['output']

X_test = test_df[['venue', 'ball', 'striker', 'bowler']]
y_test = test_df['output']

# Create and train the Gradient Boosting classifier
gb = GradientBoostingClassifier(n_estimators=100)
gb.fit(X_train, y_train)

# Test the model using the testing data and calculate the accuracy score
y_pred = gb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.44885847782280386


In [113]:
X_test['outcome'] = y_pred

C:\Users\Vishnu Kumar\AppData\Local\Temp\ipykernel_15892\1904612857.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['outcome'] = y_pred


In [116]:
X_test['outcome'].unique()

array(['1', '0', 'W', '6', '3', '7'], dtype=object)

In [121]:
for outcome in df['output'].unique():
    print ("Outcome ", outcome)
    
    test_slice = df[df["output"] == outcome]
    
    predicted_slice = X_test[X_test['outcome'] == outcome]
    
    print ("Actual Count", len(test_slice), "Predicted Count", len(predicted_slice)) 

    
#     # get rows which were labelled with this outcome in the test data
#     test_slice = test[test["outcome"] == outcome]

#     # get rows which were predicted with this outcome by the model
#     predicted_slice = predicted[predicted["outcome"] == outcome]

#     # a vector of the average values of the inputs that were labelled with the outcome in the test set
#     test_slice_mean = test_slice.mean() 

#     # a vector of the average values of the inputs that were predicted to have this outcome by the model
#     predicted_slice_mean = predicted_slice.mean().fillna(0) 

#     # calculates the euclidean distance between the two vectors
#     dist = ((predicted_slice_mean - test_slice_mean) ** 2).sum() ** 0.5

#     print ("Actual Count", len(test_slice), "Predicted Count", len(predicted_slice)) # compared how many times this outcome appeared in the test set and how many times it got predicted
#     print ("Average Distance", dist)

Outcome  1
Actual Count 93424 Predicted Count 31575
Outcome  0
Actual Count 68481 Predicted Count 13571
Outcome  4
Actual Count 25806 Predicted Count 0
Outcome  6
Actual Count 10586 Predicted Count 2
Outcome  5
Actual Count 442 Predicted Count 0
Outcome  W
Actual Count 11124 Predicted Count 6
Outcome  2
Actual Count 15030 Predicted Count 0
Outcome  3
Actual Count 836 Predicted Count 3
Outcome  7
Actual Count 64 Predicted Count 2


In [140]:
predicted = gb.predict_proba(X_test.drop(["outcome"], axis=1))

In [141]:
predicted

array([[2.19219893e-01, 4.60096722e-01, 6.83824508e-02, ...,
        6.93735621e-02, 5.03904379e-04, 6.06761345e-02],
       [2.48754794e-01, 5.19338777e-01, 7.18536788e-02, ...,
        3.45479923e-02, 7.14253996e-05, 4.07439013e-02],
       [2.74856576e-01, 4.79702127e-01, 7.51234558e-02, ...,
        4.03369626e-02, 4.61693079e-05, 3.89623554e-02],
       ...,
       [2.23557967e-01, 4.52356705e-01, 8.17302798e-02, ...,
        6.38481548e-02, 2.54762882e-04, 6.48415700e-02],
       [2.16975791e-01, 4.48071128e-01, 9.68973856e-02, ...,
        5.39372419e-02, 5.17408045e-04, 5.87816460e-02],
       [4.99917812e-01, 2.76895471e-01, 4.50710597e-02, ...,
        1.81968297e-02, 2.30394009e-05, 2.80228208e-02]])

In [142]:
preds = []

In [143]:
from random import choices

for weights in predicted:
    outcomes = ['0','1','2','3','4','5','6','7','W']
    print(weights)
    # weighted random selection
    p = choices(outcomes, weights=weights)

    preds.append(p[0])
    
    print(preds)
    break
    


[0.21921989 0.46009672 0.06838245 0.00223005 0.11789129 0.00162599
 0.06937356 0.0005039  0.06067613]
['1']


In [132]:
predicted = X_test.drop(["outcome"], axis=1)
predicted["outcome"] = preds


In [133]:
for outcome in df['output'].unique():
    print ("Outcome ", outcome)
    
    test_slice = df[df["output"] == outcome]
    
    predicted_slice = predicted[predicted['outcome'] == outcome]
    
    print ("Actual Count", len(test_slice), "Predicted Count", len(predicted_slice)) 


Outcome  1
Actual Count 93424 Predicted Count 18693
Outcome  0
Actual Count 68481 Predicted Count 13790
Outcome  4
Actual Count 25806 Predicted Count 5093
Outcome  6
Actual Count 10586 Predicted Count 2133
Outcome  5
Actual Count 442 Predicted Count 87
Outcome  W
Actual Count 11124 Predicted Count 2263
Outcome  2
Actual Count 15030 Predicted Count 2924
Outcome  3
Actual Count 836 Predicted Count 161
Outcome  7
Actual Count 64 Predicted Count 15


In [134]:
predicted

,venue,ball,striker,bowler,outcome
221012,47,100.0,183,133,6
60141,43,65.0,45,239,1
76083,22,61.0,155,109,1
98184,40,100.0,117,172,1
108691,19,87.0,309,242,4
...,...,...,...,...,...
224865,47,107.0,589,197,0
68789,43,32.0,220,265,0
34858,3,96.0,417,321,W
133190,32,97.0,192,282,1
